# Curate UAS Risk Analysis Datasets
Datasets involved in analyzing risk for Unmanned Aircraft System (UAS) sightings are as follows:
1) UAS Sightings Reports
2) Flight Logs for Spatiotemporal Analysis
3) UAS Facility Map (UASFM) Grid Files for Geospatial Analysis

As these datasets are retrieved, they will be curated and fused together and uploaded to the cloud for analysis (e.g., via pyspark)

## Setup

In [2]:
!python --version

Python 3.12.5


In [1]:
!cat requirements.txt

# Python 3.12.5
openpyxl==3.1.5


In [3]:
!python -m pip install -q -r requirements.txt

### Imports

In [1]:
import openpyxl 
import csv 
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
from zipfile import ZipFile
from uas_risk_analysis_packages.ml import functions as F

[nltk_data] Downloading package stopwords to /home/ballen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ballen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Configurations

In [2]:
DATASETS_DIR = 'datasets'
TEMPLATES_DIR = 'templates'

## Curate Datasets
**NOTE** Need to manually download the raw `xlsx` data from the FAA site.<br>https://www.faa.gov/uas/resources/public_records/uas_sightings_report

Convert raw UAS sightings reports to CSV. Later, the CSV data will be processed by a large language model (LLM) to extract spatiotemporal data (e.g., lat/lon, altitude, time)

### UAS Sightings Reports

In [4]:
uas_sightings_reports_raw_dir = os.path.join(DATASETS_DIR, 'uas_sightings_reports/raw')
uas_sightings_reports_paths = glob(os.path.join(uas_sightings_reports_raw_dir, '*.xlsx'))
uas_sightings_reports_paths

['datasets/uas_sightings_reports/raw/uas_sightings_report_fy23-q1.xlsx',
 'datasets/uas_sightings_reports/raw/UAS_Sightings_report_1Feb-31Mar16.xlsx',
 'datasets/uas_sightings_reports/raw/FY2019_Q3_UAS_Sightings.xlsx',
 'datasets/uas_sightings_reports/raw/FY2019_Q4_UAS_Sightings.xlsx',
 'datasets/uas_sightings_reports/raw/FY2022_Q1_UAS_Sightings.xlsx',
 'datasets/uas_sightings_reports/raw/UAS_Sightings_report_1Apr-30Jun16.xlsx',
 'datasets/uas_sightings_reports/raw/uas_sightings_report_fy23_q4.xlsx',
 'datasets/uas_sightings_reports/raw/FY2021_Q4_UAS_Sightings.xlsx',
 'datasets/uas_sightings_reports/raw/FY2018_Q1_UAS_Sightings.xlsx',
 'datasets/uas_sightings_reports/raw/FY2019_Q2_UAS_Sightings.xlsx',
 'datasets/uas_sightings_reports/raw/UAS_sightings_report_170331.xlsx',
 'datasets/uas_sightings_reports/raw/FY2021_Q3_UAS_Sightings.xlsx',
 'datasets/uas_sightings_reports/raw/Reported-UAS-Sigihtings-January-March-2022_0.xlsx',
 'datasets/uas_sightings_reports/raw/FY2018_Q2_UAS_Sightings.

In [6]:
uas_sightings_reports_convert_dir = os.path.join(DATASETS_DIR, 'uas_sightings_reports/convert')
uas_sightings_reports_convert_paths = []

with tqdm(total=len(uas_sightings_reports_paths)) as pbar:
    for path in uas_sightings_reports_paths:
        pbar.set_description(os.path.basename(path), refresh=True)
        pbar.update(1)

        # Set output path and make output directory if it doesn't exist yet
        outpath = os.path.join(uas_sightings_reports_convert_dir, os.path.splitext(os.path.basename(path))[0] + '.csv')
        os.makedirs(os.path.dirname(outpath), exist_ok=True)

        if not os.path.exists(outpath):
            # Open workbook and select active sheet
            workbook = openpyxl.load_workbook(path)
            sheet = workbook.active
            
            # Open the output file for writing and save the active sheet contents to it
            with open(outpath, 'w', newline='') as f:
                writer = csv.writer(f)
        
                # Iterate rows in active sheet and write to file
                for row in sheet.rows:
                    writer.writerow([cell.value for cell in row])
        
            # print(f'Saved to "{outpath}"')
        uas_sightings_reports_convert_paths.append(outpath)

uas_sightings_report_fy23-q3.xlsx: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 177.54it/s]


In [7]:
uas_sightings_reports_convert_paths

['datasets/uas_sightings_reports/convert/uas_sightings_report_fy23-q1.csv',
 'datasets/uas_sightings_reports/convert/UAS_Sightings_report_1Feb-31Mar16.csv',
 'datasets/uas_sightings_reports/convert/FY2019_Q3_UAS_Sightings.csv',
 'datasets/uas_sightings_reports/convert/FY2019_Q4_UAS_Sightings.csv',
 'datasets/uas_sightings_reports/convert/FY2022_Q1_UAS_Sightings.csv',
 'datasets/uas_sightings_reports/convert/UAS_Sightings_report_1Apr-30Jun16.csv',
 'datasets/uas_sightings_reports/convert/uas_sightings_report_fy23_q4.csv',
 'datasets/uas_sightings_reports/convert/FY2021_Q4_UAS_Sightings.csv',
 'datasets/uas_sightings_reports/convert/FY2018_Q1_UAS_Sightings.csv',
 'datasets/uas_sightings_reports/convert/FY2019_Q2_UAS_Sightings.csv',
 'datasets/uas_sightings_reports/convert/UAS_sightings_report_170331.csv',
 'datasets/uas_sightings_reports/convert/FY2021_Q3_UAS_Sightings.csv',
 'datasets/uas_sightings_reports/convert/Reported-UAS-Sigihtings-January-March-2022_0.csv',
 'datasets/uas_sightin

In [8]:
# Ensure headers are the same for each UAS sightings report (most likely not the case)
# Date, State, City, Summary
# ==> datasets/uas_sightings_reports/convert/UAS_Sightings_report_21Aug-31Jan.csv <==
# EventDATETIME,EventREPORTNARRATIVE,LocationCITY,LocationSTATE

# ==> datasets/uas_sightings_reports/convert/UAS_Sightings_report_1Jul-30Sep16.csv <==
# Event Date & Time,CITY,STATE,Event Description

# ==> datasets/uas_sightings_reports/convert/UAS_sightings_report_170331.csv <==
# Event Date,CITY,STATE,Event Description,Column1,Column2,Column3,Column4

# ==> datasets/uas_sightings_reports/convert/UAS_Sightings_report_1Apr-30Jun16.csv <==
# spEventDateTime,spCity,spState,Redacted

# ==> datasets/uas_sightings_reports/convert/UAS_Sightings_report_1Feb-31Mar16.csv <==
# Event DATETIME,Description,Location CITY,Location STATE

# ==> datasets/uas_sightings_reports/convert/FY2017_Q4_De-identification_Redaction_11192017.csv <==
# Event Date,Event Description,City,State

# ==> datasets/uas_sightings_reports/convert/FY2018_Q1_UAS_Sightings.csv <==
# Date of Sighting,City,State,Summary

# ==> datasets/uas_sightings_reports/convert/FY2018_Q2_UAS_Sightings.csv <==
# Date of Sighting,City,State,Summary

# !head -1 {os.path.join(uas_sightings_reports_convert_dir, '*.csv')}
for path in glob(os.path.join(uas_sightings_reports_convert_dir, '*.csv')):
    with open(path, 'r') as f:
        line = f.readline()
        print(f'==> {path} <==')
        if len(line) >= 100:
            print(line[:50] + ' ... ' + line[-50:])
        else:
            print(line)

==> datasets/uas_sightings_reports/convert/UAS_sightings_report_Apr_Jun2017.csv <==
Date of Sighting,State,City,Summary

==> datasets/uas_sightings_reports/convert/uas_sightings_report_fy23-q3.csv <==
Date,State,City,Summary

==> datasets/uas_sightings_reports/convert/FY2019_Q2_UAS_Sightings.csv <==
Date of Sighting,State,City,Summary

==> datasets/uas_sightings_reports/convert/Reported-UAS-Sightings-July-Sept-2022.csv <==
Date,State,City,Summary

==> datasets/uas_sightings_reports/convert/FY2022_Q1_UAS_Sightings.csv <==
Day of Sighting,State,City,Summary

==> datasets/uas_sightings_reports/convert/uas_sightings_report_fy24_q1.csv <==
Date,State,City,Summary

==> datasets/uas_sightings_reports/convert/uas_sightings_report_fy24_q2.csv <==
Date,State,City,Summary

==> datasets/uas_sightings_reports/convert/FY2020_Q3_UAS_Sightings.csv <==
Day of Sighting,State,City,Summary

==> datasets/uas_sightings_reports/convert/UAS_Sightings_report_21Aug-31Jan.csv <==
EventDATETIME,EventREPORTNARRATI

We can see there are a lot of discrepancies with the column names. Two of the files, `datasets/uas_sightings_reports/convert/uas_sightings_report_fy23-q1.csv` and `datasets/uas_sightings_reports/convert/uas_sightings_report_fy23-q2.csv`, appear to have a corrupt headers with over 16k columns.

The approach for applying a common schema is to define a set of columns that appear to be the most common and map the actual columns to those common columns using similarity checks.

In [ ]:
common_cols = [
    'date',
    'city',
    'state',
    'summary',
    'narrative',
    'description',
]

### Flight Logs (OpenSky API)

### UASFM Grids
See: https://uas-faa.opendata.arcgis.com/datasets/faa::faa-uas-facilitymap-data/about

In [17]:
uasfm_grids_dir = os.path.join(DATASETS_DIR, 'uasfm_grids')
with ZipFile(os.path.join(uasfm_grids_dir, 'FAA_UAS_FacilityMap_Data.zip')) as zf:
    zf.extractall(uasfm_grids_dir)